In [ ]:
import cv2
import dlib
import numpy as np

# 얼굴 랜드마크 모델 로드
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("C:\Users\AI-LHJ\Desktop\PC Project\detection\shape_predictor_68_face_landmarks.dat")

def extract_cheek_mean_color(image_path):
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    faces = detector(img_rgb)
    if len(faces) == 0:
        return None
    face = faces[0]
    landmarks = predictor(img_rgb, face)
    # 볼(cheek) 영역 좌표 예시 (landmark 36~48 사이 영역 등)
    left_cheek = np.array([(landmarks.part(1).x, landmarks.part(1).y),
                           (landmarks.part(2).x, landmarks.part(2).y),
                           (landmarks.part(3).x, landmarks.part(3).y),
                           (landmarks.part(4).x, landmarks.part(4).y),
                           (landmarks.part(31).x, landmarks.part(31).y),
                           (landmarks.part(48).x, landmarks.part(48).y)])
    mask = np.zeros(img_rgb.shape[:2], dtype=np.uint8)
    cv2.fillPoly(mask, [left_cheek], 255)
    mean_color = cv2.mean(img_rgb, mask=mask)[:3]  # (R, G, B)
    return mean_color  # (R, G, B)-


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (3768951888.py, line 7)

In [3]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import os

# 퍼스널 컬러 클래스 이름 (모델에 맞게 수정 필요)
personal_color_classes = ['봄 웜톤', '여름 쿨톤', '가을 웜톤', '겨울 쿨톤']

# h5 모델 파일 경로 (실제 파일 경로로 변경 필요)
model_path = 'C:/Users/AI-LHJ/Desktop/PC Project/analysis/PersonalColorAnalyzerModel/personal_color.h5'

def load_personal_color_model(model_path):
    """퍼스널 컬러 모델을 로드합니다."""
    try:
        model = load_model(model_path)
        print("모델이 성공적으로 로드되었습니다.")
        return model
    except Exception as e:
        print(f"모델 로드 중 오류가 발생했습니다: {str(e)}")
        return None

def preprocess_image(img_path, target_size=(128, 128)):
    """이미지를 전처리합니다."""
    try:
        # 이미지가 경로인 경우
        if isinstance(img_path, str):
            if not os.path.exists(img_path):
                print(f"이미지 파일이 존재하지 않습니다: {img_path}")
                return None
            img = cv2.imread(img_path)
            if img is None:
                print(f"이미지를 읽을 수 없습니다: {img_path}")
                return None
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # 이미지가 이미 배열인 경우
        else:
            img = img_path.copy()
            if len(img.shape) == 2:  # 그레이스케일 이미지인 경우
                img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
            elif img.shape[2] == 4:  # RGBA 이미지인 경우
                img = cv2.cvtColor(img, cv2.COLOR_RGBA2RGB)
            
        # 이미지 크기 조정
        img_resized = cv2.resize(img, target_size)
        
        # 이미지를 모델 입력에 맞게 정규화 (0~1 범위로)
        img_normalized = img_resized.astype(np.float32) / 255.0
        
        # 배치 차원 추가
        img_array = np.expand_dims(img_normalized, axis=0)
        
        return img_array, img
    
    except Exception as e:
        print(f"이미지 전처리 중 오류가 발생했습니다: {str(e)}")
        return None, None

def detect_face(img):
    """이미지에서 얼굴을 감지합니다."""
    try:
        # OpenCV의 얼굴 탐지기 로드
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        
        # 그레이스케일 변환
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        
        # 얼굴 감지
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        
        if len(faces) > 0:
            # 가장 큰 얼굴 찾기
            max_area = 0
            max_face = None
            for (x, y, w, h) in faces:
                if w*h > max_area:
                    max_area = w*h
                    max_face = (x, y, w, h)
            
            # 얼굴 영역 추출
            x, y, w, h = max_face
            face_img = img[y:y+h, x:x+w]
            return face_img, max_face
        else:
            print("이미지에서 얼굴을 찾을 수 없습니다.")
            return None, None
    
    except Exception as e:
        print(f"얼굴 감지 중 오류가 발생했습니다: {str(e)}")
        return None, None

def predict_personal_color(model, img_path, use_face_detection=True):
    """이미지의 퍼스널 컬러를 예측합니다."""
    # 이미지 전처리
    img_array, original_img = preprocess_image(img_path)
    if img_array is None:
        return None, None, None
    
    face_rect = None
    # 얼굴 감지 옵션이 켜져 있으면 얼굴 부분만 추출
    if use_face_detection:
        face_img, face_rect = detect_face(original_img)
        if face_img is not None:
            # 얼굴 이미지 전처리
            img_array, _ = preprocess_image(face_img)
    
    # 예측 수행
    try:
        predictions = model.predict(img_array)
        predicted_class_index = np.argmax(predictions[0])
        confidence = predictions[0][predicted_class_index]
        
        # 클래스 이름이 있으면 사용, 없으면 인덱스 반환
        if len(personal_color_classes) > predicted_class_index:
            predicted_class = personal_color_classes[predicted_class_index]
        else:
            predicted_class = f"Class {predicted_class_index}"
        
        return predicted_class, confidence, face_rect
    
    except Exception as e:
        print(f"예측 중 오류가 발생했습니다: {str(e)}")
        return None, None, None

def visualize_result(img_path, predicted_class, confidence, face_rect=None):
    """예측 결과를 시각화합니다."""
    try:
        # 이미지가 경로인 경우
        if isinstance(img_path, str):
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        else:
            img = img_path.copy()
        
        plt.figure(figsize=(10, 6))
        
        # 얼굴 영역 표시
        if face_rect is not None:
            x, y, w, h = face_rect
            cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        plt.imshow(img)
        plt.title(f'퍼스널 컬러 예측: {predicted_class} (확률: {confidence:.2f})')
        plt.axis('off')
        plt.show()
    
    except Exception as e:
        print(f"결과 시각화 중 오류가 발생했습니다: {str(e)}")

def main():
    # 모델 로드
    model = load_personal_color_model(model_path)
    if model is None:
        return
    
    # 이미지 경로 입력 받기
    img_path = input("분석할 이미지 경로를 입력해주세요: ")
    
    # 얼굴 감지 사용 여부
    use_face_detection = input("얼굴 감지를 사용하시겠습니까? (y/n): ").lower() == 'y'
    
    # 퍼스널 컬러 예측
    predicted_class, confidence, face_rect = predict_personal_color(model, img_path, use_face_detection)
    
    if predicted_class is not None:
        print(f"예측된 퍼스널 컬러: {predicted_class}")
        print(f"예측 확률: {confidence:.2f}")
        
        # 결과 시각화
        visualize_result(img_path, predicted_class, confidence, face_rect)
    else:
        print("퍼스널 컬러 예측에 실패했습니다.")

if __name__ == "__main__":
    main()

모델이 성공적으로 로드되었습니다.
이미지 파일이 존재하지 않습니다: 0


TypeError: cannot unpack non-iterable NoneType object

In [1]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import os
import urllib.request
from urllib.parse import urlparse
import argparse
import time

# 퍼스널 컬러 클래스 이름 (모델에 맞게 수정 필요)
personal_color_classes = ['봄 웜톤', '여름 쿨톤', '가을 웜톤', '겨울 쿨톤']

# 모델 입력 크기 (모델에 맞게 수정 필요)
MODEL_INPUT_SIZE = (128, 128)

def load_personal_color_model(model_path):
    """퍼스널 컬러 모델을 로드합니다."""
    try:
        model = load_model(model_path)
        print("모델이 성공적으로 로드되었습니다.")
        return model
    except Exception as e:
        print(f"모델 로드 중 오류가 발생했습니다: {str(e)}")
        return None

def load_image_from_source(source):
    """다양한 소스(파일 경로, URL, 배열)에서 이미지를 로드합니다."""
    img = None
    try:
        # URL에서 이미지 로드
        if isinstance(source, str) and (source.startswith('http://') or source.startswith('https://')):
            try:
                parsed_url = urlparse(source)
                filename = os.path.basename(parsed_url.path)
                temp_path = f"temp_{int(time.time())}_{filename}"
                
                # URL에서 이미지 다운로드
                urllib.request.urlretrieve(source, temp_path)
                img = cv2.imread(temp_path)
                
                # 임시 파일 삭제
                if os.path.exists(temp_path):
                    os.remove(temp_path)
                    
                if img is None:
                    raise Exception("URL에서 이미지를 로드할 수 없습니다")
                
            except Exception as e:
                print(f"URL에서 이미지 로드 중 오류 발생: {str(e)}")
                return None
                
        # 파일 경로에서 이미지 로드
        elif isinstance(source, str):
            if not os.path.exists(source):
                print(f"이미지 파일이 존재하지 않습니다: {source}")
                return None
                
            img = cv2.imread(source)
            if img is None:
                print(f"이미지를 읽을 수 없습니다: {source}")
                return None
                
        # 이미 배열인 경우 (예: 웹캠에서 프레임 캡처)
        elif isinstance(source, np.ndarray):
            img = source.copy()
            
        else:
            print("지원되지 않는 이미지 소스입니다.")
            return None
            
        # BGR을 RGB로 변환 (OpenCV는 BGR로 로드함)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        return img
        
    except Exception as e:
        print(f"이미지 로드 중 오류가 발생했습니다: {str(e)}")
        return None

def preprocess_image(img, target_size=MODEL_INPUT_SIZE):
    """이미지를 전처리합니다."""
    try:
        if img is None:
            return None
            
        # 이미지 형식 처리
        if len(img.shape) == 2:  # 그레이스케일 이미지인 경우
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        elif img.shape[2] == 4:  # RGBA 이미지인 경우
            img = cv2.cvtColor(img, cv2.COLOR_RGBA2RGB)
            
        # 이미지 크기 조정
        img_resized = cv2.resize(img, target_size)
        
        # 이미지를 모델 입력에 맞게 정규화 (0~1 범위로)
        img_normalized = img_resized.astype(np.float32) / 255.0
        
        # 배치 차원 추가
        img_array = np.expand_dims(img_normalized, axis=0)
        
        return img_array
    
    except Exception as e:
        print(f"이미지 전처리 중 오류가 발생했습니다: {str(e)}")
        return None

def detect_face(img):
    """이미지에서 얼굴을 감지합니다."""
    try:
        if img is None:
            return None, None
            
        # OpenCV의 얼굴 탐지기 로드
        face_cascade_path = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
        
        # 얼굴 탐지기 파일이 존재하는지 확인
        if not os.path.exists(face_cascade_path):
            print(f"얼굴 탐지기 파일을 찾을 수 없습니다: {face_cascade_path}")
            # 대체 경로 시도
            alt_paths = [
                'haarcascade_frontalface_default.xml',
                './haarcascade_frontalface_default.xml',
                '../haarcascade_frontalface_default.xml'
            ]
            
            for path in alt_paths:
                if os.path.exists(path):
                    face_cascade_path = path
                    break
            else:
                print("얼굴 탐지기 파일을 찾을 수 없어 얼굴 감지를 건너뜁니다.")
                return None, None
        
        face_cascade = cv2.CascadeClassifier(face_cascade_path)
        
        # 그레이스케일 변환
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        
        # 얼굴 감지
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        
        if len(faces) > 0:
            # 가장 큰 얼굴 찾기
            max_area = 0
            max_face = None
            for (x, y, w, h) in faces:
                if w*h > max_area:
                    max_area = w*h
                    max_face = (x, y, w, h)
            
            # 얼굴 영역 추출
            x, y, w, h = max_face
            
            # 얼굴 주변 영역을 좀 더 포함하기
            margin = int(min(w, h) * 0.2)  # 20% 마진
            x_start = max(0, x - margin)
            y_start = max(0, y - margin)
            x_end = min(img.shape[1], x + w + margin)
            y_end = min(img.shape[0], y + h + margin)
            
            face_img = img[y_start:y_end, x_start:x_end]
            face_rect = (x_start, y_start, x_end - x_start, y_end - y_start)
            
            return face_img, face_rect
        else:
            print("이미지에서 얼굴을 찾을 수 없습니다. 전체 이미지를 사용합니다.")
            return None, None
    
    except Exception as e:
        print(f"얼굴 감지 중 오류가 발생했습니다: {str(e)}")
        return None, None

def predict_personal_color(model, img_source, use_face_detection=True):
    """이미지의 퍼스널 컬러를 예측합니다."""
    # 이미지 로드
    original_img = load_image_from_source(img_source)
    if original_img is None:
        return None, None, None, None
    
    face_img = None
    face_rect = None
    
    # 얼굴 감지 옵션이 켜져 있으면 얼굴 부분만 추출
    if use_face_detection:
        face_img, face_rect = detect_face(original_img)
    
    # 얼굴이 감지되면 얼굴 이미지 사용, 아니면 원본 이미지 사용
    img_to_predict = face_img if face_img is not None else original_img
    
    # 이미지 전처리
    img_array = preprocess_image(img_to_predict)
    if img_array is None:
        return None, None, None, None
    
    # 예측 수행
    try:
        predictions = model.predict(img_array)
        predicted_class_index = np.argmax(predictions[0])
        confidence = float(predictions[0][predicted_class_index])
        
        # 모든 클래스에 대한 확률 수집
        all_probabilities = {
            personal_color_classes[i] if i < len(personal_color_classes) else f"Class {i}": float(predictions[0][i])
            for i in range(len(predictions[0]))
        }
        
        # 클래스 이름이 있으면 사용, 없으면 인덱스 반환
        if predicted_class_index < len(personal_color_classes):
            predicted_class = personal_color_classes[predicted_class_index]
        else:
            predicted_class = f"Class {predicted_class_index}"
        
        return predicted_class, confidence, face_rect, all_probabilities
    
    except Exception as e:
        print(f"예측 중 오류가 발생했습니다: {str(e)}")
        return None, None, None, None

def visualize_result(img_source, predicted_class, confidence, face_rect=None, all_probabilities=None):
    """예측 결과를 시각화합니다."""
    try:
        # 이미지 로드
        img = load_image_from_source(img_source)
        if img is None:
            return
        
        # Figure 설정
        plt.figure(figsize=(14, 7))
        
        # 이미지 표시 (왼쪽)
        plt.subplot(1, 2, 1)
        
        # 얼굴 영역 표시
        if face_rect is not None:
            x, y, w, h = face_rect
            cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        plt.imshow(img)
        plt.title(f'퍼스널 컬러 예측: {predicted_class} (확률: {confidence:.2f})')
        plt.axis('off')
        
        # 확률 바 그래프 표시 (오른쪽)
        if all_probabilities is not None:
            plt.subplot(1, 2, 2)
            
            colors = ['#FFB6C1', '#ADD8E6', '#D2B48C', '#E6E6FA']  # 각 계절에 맞는 색상
            classes = list(all_probabilities.keys())
            values = list(all_probabilities.values())
            
            y_pos = np.arange(len(classes))
            
            # 수평 바 차트 생성
            bars = plt.barh(y_pos, values, color=colors[:len(classes)])
            
            # 바 위에 값 표시
            for i, v in enumerate(values):
                plt.text(v + 0.01, i, f'{v:.2f}', va='center')
            
            plt.yticks(y_pos, classes)
            plt.xlabel('확률')
            plt.title('퍼스널 컬러 예측 확률')
            
            # 확률이 가장 높은 바 강조
            max_idx = values.index(max(values))
            bars[max_idx].set_edgecolor('black')
            bars[max_idx].set_linewidth(2)
        
        plt.tight_layout()
        plt.show()
    
    except Exception as e:
        print(f"결과 시각화 중 오류가 발생했습니다: {str(e)}")

def capture_from_webcam(model):
    """웹캠에서 이미지를 캡처하여 퍼스널 컬러를 분석합니다."""
    try:
        cap = cv2.VideoCapture(0)
        
        if not cap.isOpened():
            print("웹캠을 열 수 없습니다.")
            return
        
        print("웹캠이 준비되었습니다. 스페이스바를 눌러 사진을 촬영하세요. 'q'를 눌러 종료하세요.")
        
        while True:
            ret, frame = cap.read()
            
            if not ret:
                print("프레임을 캡처할 수 없습니다.")
                break
            
            # 화면에 프레임 표시
            cv2.imshow('Webcam (Press SPACE to capture, Q to quit)', frame)
            
            key = cv2.waitKey(1) & 0xFF
            
            # 스페이스바를 누르면 캡처
            if key == 32:  # 스페이스바
                # 이미지 분석
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                predicted_class, confidence, face_rect, all_probabilities = predict_personal_color(model, rgb_frame)
                
                if predicted_class is not None:
                    print(f"예측된 퍼스널 컬러: {predicted_class}")
                    print(f"예측 확률: {confidence:.2f}")
                    
                    # 결과 시각화
                    visualize_result(rgb_frame, predicted_class, confidence, face_rect, all_probabilities)
                else:
                    print("퍼스널 컬러 예측에 실패했습니다.")
            
            # 'q'를 누르면 종료
            if key == ord('q'):
                break
        
        cap.release()
        cv2.destroyAllWindows()
    
    except Exception as e:
        print(f"웹캠 캡처 중 오류가 발생했습니다: {str(e)}")
        if 'cap' in locals() and cap.isOpened():
            cap.release()
        cv2.destroyAllWindows()

def batch_process_images(model, directory_path, use_face_detection=True):
    """디렉토리 내의 모든 이미지를 일괄 처리합니다."""
    try:
        if not os.path.exists(directory_path) or not os.path.isdir(directory_path):
            print(f"디렉토리가 존재하지 않습니다: {directory_path}")
            return
        
        # 지원하는 이미지 확장자
        image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.gif', '.webp']
        
        # 디렉토리 내 모든 파일 검색
        image_files = []
        for root, _, files in os.walk(directory_path):
            for file in files:
                if any(file.lower().endswith(ext) for ext in image_extensions):
                    image_files.append(os.path.join(root, file))
        
        if not image_files:
            print(f"디렉토리에 이미지 파일이 없습니다: {directory_path}")
            return
        
        print(f"총 {len(image_files)}개의 이미지 파일을 처리합니다...")
        
        results = []
        
        for img_path in image_files:
            print(f"처리 중: {img_path}")
            
            predicted_class, confidence, face_rect, all_probabilities = predict_personal_color(
                model, img_path, use_face_detection
            )
            
            if predicted_class is not None:
                results.append({
                    'file': img_path,
                    'predicted_class': predicted_class,
                    'confidence': confidence
                })
                print(f"예측된 퍼스널 컬러: {predicted_class} (확률: {confidence:.2f})")
            else:
                print("퍼스널 컬러 예측에 실패했습니다.")
        
        # 결과 요약 출력
        print("\n===== 처리 결과 요약 =====")
        
        if results:
            for result in results:
                print(f"파일: {result['file']}")
                print(f"예측된 퍼스널 컬러: {result['predicted_class']}")
                print(f"확률: {result['confidence']:.2f}")
                print("------------------------")
            
            # 결과를 CSV 파일로 저장할지 묻기
            save_csv = input("결과를 CSV 파일로 저장하시겠습니까? (y/n): ").lower() == 'y'
            
            if save_csv:
                import csv
                csv_path = os.path.join(directory_path, "personal_color_results.csv")
                
                with open(csv_path, 'w', newline='', encoding='utf-8') as csvfile:
                    fieldnames = ['파일', '퍼스널 컬러', '확률']
                    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                    
                    writer.writeheader()
                    for result in results:
                        writer.writerow({
                            '파일': result['file'],
                            '퍼스널 컬러': result['predicted_class'],
                            '확률': f"{result['confidence']:.2f}"
                        })
                
                print(f"결과가 CSV 파일로 저장되었습니다: {csv_path}")
        else:
            print("처리된 결과가 없습니다.")
    
    except Exception as e:
        print(f"일괄 처리 중 오류가 발생했습니다: {str(e)}")

def main(model_path='C:/Users/AI-LHJ/Desktop/PC Project/analysis/PersonalColorAnalyzerModel/best_model_resnet_ALL.pth', mode=None, source=None, use_face_detection=True):
    """
    퍼스널 컬러 분류 프로그램의 메인 함수
    
    매개변수:
    - model_path: h5 모델 파일 경로
    - mode: 동작 모드 ('image', 'url', 'webcam', 'batch' 중 하나)
    - source: 이미지 경로, URL 또는 디렉토리 경로
    - use_face_detection: 얼굴 감지 사용 여부
    """
    # Jupyter에서 실행되는 경우 argparse 대신 함수 매개변수를 직접 사용
    # 명령줄 환경인 경우에만 argparse 사용
    import sys
    is_jupyter = any(arg.endswith('json') for arg in sys.argv)
    
    if not is_jupyter and len(sys.argv) > 1 and not sys.argv[0].endswith('ipykernel_launcher.py'):
        try:
            parser = argparse.ArgumentParser(description='퍼스널 컬러 분류 프로그램')
            
            # 필수 인자
            parser.add_argument('--model', type=str, default='personal_color_model.h5',
                                help='H5 모델 파일 경로 (기본값: personal_color_model.h5)')
            
            # 옵션 인자
            group = parser.add_mutually_exclusive_group()
            group.add_argument('--image', type=str, help='분석할 이미지 파일 경로')
            group.add_argument('--url', type=str, help='분석할 이미지의 URL')
            group.add_argument('--webcam', action='store_true', help='웹캠 사용 모드')
            group.add_argument('--batch', type=str, help='일괄 처리할 이미지가 있는 디렉토리 경로')
            
            parser.add_argument('--no-face', action='store_true', help='얼굴 감지를 사용하지 않음 (전체 이미지 사용)')
            
            args = parser.parse_args()
            
            # argparse 인자를 함수 매개변수로 변환
            model_path = args.model
            use_face_detection = not args.no_face
            
            if args.image:
                mode = 'image'
                source = args.image
            elif args.url:
                mode = 'url'
                source = args.url
            elif args.webcam:
                mode = 'webcam'
            elif args.batch:
                mode = 'batch'
                source = args.batch
        except Exception as e:
            print(f"명령줄 인자 처리 중 오류 발생: {str(e)}")
            print("대화형 모드로 전환합니다.")
    
    # 모델 로드
    model = load_personal_color_model(model_path)
    if model is None:
        return
    
    # 웹캠 모드
    if mode == 'webcam':
        capture_from_webcam(model)
        return
    
    # 배치 처리 모드
    if mode == 'batch':
        batch_process_images(model, source, use_face_detection)
        return
    
    # 이미지 소스 결정
    img_source = None
    if mode == 'image':
        img_source = source
    elif mode == 'url':
        img_source = source
    else:
        # 인자가 주어지지 않은 경우 대화형 모드 시작
        print("대화형 모드를 시작합니다.")
        
        while True:
            print("\n===== 퍼스널 컬러 분석기 =====")
            print("1. 이미지 파일 분석")
            print("2. 이미지 URL 분석")
            print("3. 웹캠으로 촬영 및 분석")
            print("4. 디렉토리 내 이미지 일괄 분석")
            print("0. 종료")
            
            choice = input("원하는 작업을 선택하세요: ")
            
            if choice == '0':
                print("프로그램을 종료합니다.")
                break
            elif choice == '1':
                img_path = input("분석할 이미지 경로를 입력해주세요: ")
                use_face = input("얼굴 감지를 사용하시겠습니까? (y/n, 기본값: y): ").lower() != 'n'
                
                predicted_class, confidence, face_rect, all_probabilities = predict_personal_color(
                    model, img_path, use_face
                )
                
                if predicted_class is not None:
                    print(f"예측된 퍼스널 컬러: {predicted_class}")
                    print(f"예측 확률: {confidence:.2f}")
                    
                    # 결과 시각화
                    visualize_result(img_path, predicted_class, confidence, face_rect, all_probabilities)
                else:
                    print("퍼스널 컬러 예측에 실패했습니다.")
            elif choice == '2':
                img_url = input("분석할 이미지 URL을 입력해주세요: ")
                use_face = input("얼굴 감지를 사용하시겠습니까? (y/n, 기본값: y): ").lower() != 'n'
                
                predicted_class, confidence, face_rect, all_probabilities = predict_personal_color(
                    model, img_url, use_face
                )
                
                if predicted_class is not None:
                    print(f"예측된 퍼스널 컬러: {predicted_class}")
                    print(f"예측 확률: {confidence:.2f}")
                    
                    # 결과 시각화
                    visualize_result(img_url, predicted_class, confidence, face_rect, all_probabilities)
                else:
                    print("퍼스널 컬러 예측에 실패했습니다.")
            elif choice == '3':
                capture_from_webcam(model)
            elif choice == '4':
                dir_path = input("분석할 이미지가 있는 디렉토리 경로를 입력해주세요: ")
                use_face = input("얼굴 감지를 사용하시겠습니까? (y/n, 기본값: y): ").lower() != 'n'
                
                batch_process_images(model, dir_path, use_face)
            else:
                print("잘못된 선택입니다. 다시 시도해주세요.")
        
        return
    
    # 이미지 분석 (이미지 경로나 URL이 제공된 경우)
    if img_source:
        predicted_class, confidence, face_rect, all_probabilities = predict_personal_color(
            model, img_source, use_face_detection
        )
        
        if predicted_class is not None:
            print(f"예측된 퍼스널 컬러: {predicted_class}")
            print(f"예측 확률: {confidence:.2f}")
            
            # 결과 시각화
            visualize_result(img_source, predicted_class, confidence, face_rect, all_probabilities)
        else:
            print("퍼스널 컬러 예측에 실패했습니다.")

if __name__ == "__main__":
    main()

모델 로드 중 오류가 발생했습니다: Unable to synchronously open file (file signature not found)
